In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing required libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split

## Getting data

In [ ]:
train_data = pd.read_csv('../input/spaceship-titanic/train.csv')
test_data = pd.read_csv('../input/spaceship-titanic/test.csv') 

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.shape,test_data.shape

In [ ]:
train_data.info()

In [ ]:
test_data.info()

## Data cleaning

In [ ]:
train_data.isna().sum()

In [ ]:
test_data.isna().sum()

In [ ]:
train_data =train_data.fillna(method= "ffill")
test_data =test_data.fillna(method= "ffill")

In [ ]:
train_data.isna().sum()

In [ ]:
test_data.isna().sum()

## Exploratory data analysis

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profrep = ProfileReport(train_data, explorative = True)

In [ ]:
profrep

In [ ]:
! pip install sweetviz

In [ ]:
import sweetviz as sv

In [ ]:
report = sv.compare(train_data,test_data,'Age')
report.show_notebook()

In [ ]:
report.show_html('age_comparison')

## Handling Categorical columns

In [ ]:
train_data['Transported'] = train_data['Transported'].astype('O')

In [ ]:
cat = []
num = []
for i in train_data.columns:
    if train_data[i].dtypes == 'O':
        cat.append(i)
    else:
        num.append(i)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in cat:
    train_data[i] = le.fit_transform(train_data[i])

In [ ]:
train_data.head()

In [ ]:
for i in cat:
    if i!='Transported':
        test_data[i] = le.fit_transform(test_data[i])
test_data.head()

In [ ]:
x_train = train_data.drop('Transported',1)
y_train = train_data.Transported

In [ ]:
x_train

In [ ]:
y_train

## Installing AutoDL libraries (APIs) for tuning Dl pipelines

In [ ]:
! pip install git+https://github.com/keras-team/keras-tuner.git
! pip install autokeras

## Model 1

In [ ]:
import autokeras as ak
from xgboost import XGBClassifier

In [ ]:
m1 = ak.StructuredDataClassifier(max_trials=5, overwrite=True)
m1.fit(x_train, y_train, epochs=20)

In [ ]:
print(m1.evaluate(x_train, y_train))

## Model 2 (XGBoost)

In [ ]:
m2 = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=-1,cv=ak)

In [ ]:
m2.fit(x_train,y_train)

In [ ]:
m2.score(x_train,y_train)

In [ ]:
pred = m2.predict(test_data)

In [ ]:
sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
sub['Transported'] = pred
sub['Transported'] = sub['Transported'].replace({1:True,0:False})
sub.set_index('PassengerId',inplace=True)
sub.head()

In [ ]:
sub.to_csv('submission.csv')